In [1]:
import pickle
import pandas as pd
import time
import json
import requests
import re
from lxml import html
from bs4 import BeautifulSoup
import csv
import tqdm
from urllib.parse import urljoin
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

options = Options()
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')
options.add_argument('start-maximized')
options.add_argument('window-size=1920x1080')  # Indstil en fast vinduesstørrelse

In [2]:
links = []  # Initialiserer en tom liste til at gemme de genererede links

# For-løkke, der itererer gennem værdierne
for offset in range(0, 1001*10, 10):  
    # Genererer en URL med en specifik offset-værdi for hver iteration
    url = f'https://www.berlingske.dk/search?query=c20&limit=10&offset={offset}'
    links.append(url)  # Tilføjer den genererede URL til listen 'links'

links  # Viser listen med alle de genererede links

['https://www.berlingske.dk/search?query=c20&limit=10&offset=0',
 'https://www.berlingske.dk/search?query=c20&limit=10&offset=10',
 'https://www.berlingske.dk/search?query=c20&limit=10&offset=20',
 'https://www.berlingske.dk/search?query=c20&limit=10&offset=30',
 'https://www.berlingske.dk/search?query=c20&limit=10&offset=40',
 'https://www.berlingske.dk/search?query=c20&limit=10&offset=50',
 'https://www.berlingske.dk/search?query=c20&limit=10&offset=60',
 'https://www.berlingske.dk/search?query=c20&limit=10&offset=70',
 'https://www.berlingske.dk/search?query=c20&limit=10&offset=80',
 'https://www.berlingske.dk/search?query=c20&limit=10&offset=90',
 'https://www.berlingske.dk/search?query=c20&limit=10&offset=100',
 'https://www.berlingske.dk/search?query=c20&limit=10&offset=110',
 'https://www.berlingske.dk/search?query=c20&limit=10&offset=120',
 'https://www.berlingske.dk/search?query=c20&limit=10&offset=130',
 'https://www.berlingske.dk/search?query=c20&limit=10&offset=140',
 'http

In [3]:
base_url = 'https://www.berlingske.dk'
article_links = []

# Brug tqdm til at tracke tiden for hver iteration
for url in tqdm.tqdm(links):
    response = requests.get(url, 
                            headers={'name': 'Amanda Grønlund', 'email': 'cws883@alumni.ku.dk'})
    
    # Parse HTML med det samme uden at gemme i en liste
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find alle links (a-tags)
    for link in soup.find_all('a', href=True):
        href = link['href']
        # Filter efter links, der indeholder "/virksomheder/" eller et andet mønster, du ønsker at matche
        if '/business/' in href or '/virksomheder/' in href or '/oekonomi/' in href or '/aktier/' in href or '/dinepenge/' in href or '/arbejdsliv/' in href or '/politik/' in href or '/indland/' in href or '/internationalt/' in href:  # Tilpas dette til det faktiske mønster af artikel-links
            full_url = urljoin(base_url, href)
            article_links.append(full_url)
    
    time.sleep(0.5)  # Vent 0,5 sekund for at undgå at overbelaste serveren

# Fjern eventuelle duplikater fra article_links
article_links = list(set(article_links))

100%|██████████████████████████████████████████████████████████████████████████████| 1001/1001 [12:38<00:00,  1.32it/s]


In [4]:
article_links

['https://www.berlingske.dk/virksomheder/dsv-vi-arbejder-konstant-paa-opkoeb',
 'https://www.berlingske.dk/virksomheder/finans-tv-graesk-zorbas-paa-det-europaeiske-aktiemarked',
 'https://www.berlingske.dk/aktier/bogen-der-ryster-wall-street-og-storsaelger-i-usa',
 'https://www.berlingske.dk/aktier/grus-i-maskineriet-til-fls-aktionaererne',
 'https://www.berlingske.dk/aktier/sverige-oedelagde-dagen-for-nordea',
 'https://www.berlingske.dk/oekonomi/det-eneste-sikre-er-usikkerheden',
 'https://www.berlingske.dk/virksomheder/ni-liv-var-ikke-nok-for-skats-topdirektoer',
 'https://www.berlingske.dk/business/bestikkelse-korruption-og-milliardoverskud-her-er-den-franske-gigant-der-har',
 'https://www.berlingske.dk/aktier/danskerne-clinton-er-bedst-for-aktiemarkedet',
 'https://www.berlingske.dk/virksomheder/darzalex-salget-falder-lidt-men-holder-stadig-hoejt-niveau',
 'https://www.berlingske.dk/aktier/torsdagens-aktier-pandoras-genrejsning-sikrede-c20-rekord',
 'https://www.berlingske.dk/akti

In [5]:
# Konfigurer Selenium WebDriver med Chrome
options = Options()
options.page_load_strategy = 'eager'  # Skift til 'eager' for hurtigere load
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Sæt implicit ventetid
driver.implicitly_wait(15)

# Dine login oplysninger
login = "jvgronlund@yahoo.com"
password = "rebklemmer72"

# Gå til Berlingskes login-side
driver.get('https://www.berlingske.dk/login')

# Afvis cookie-popup (hvis den er der)
try:
    accept_cookies_button = driver.find_element(By.CSS_SELECTOR, 'button[id="CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"]')
    accept_cookies_button.click()
    time.sleep(5)
except Exception as e:
    print("Ingen cookie-popup fundet, fortsætter uden.")

# Find login-felterne og indtast login-oplysninger
email_field = driver.find_element(By.ID, 'email')
password_field = driver.find_element(By.ID, 'current-password')

email_field.send_keys(login)
password_field.send_keys(password)

# Find og klik på login-knappen
login_button = driver.find_element(By.CSS_SELECTOR, 'button[type="submit"]')
login_button.click()

# Vent lidt for at sikre login-succes
time.sleep(5)

# Gå til Berlingske søgeresultater for C20
driver.get('https://www.berlingske.dk/search?query=c20')

# Vent lidt for at sikre, at siden er indlæst
time.sleep(5)

In [6]:
# Tom liste til at gemme artikeldetaljer
articles = []

# Loop gennem alle artikel-links
for link in tqdm.tqdm(article_links):
    try:
        # Sæt en timeout for sideindlæsning
        driver.set_page_load_timeout(60)
        
        # Brug Selenium driver til at navigere til siden
        driver.get(link)

        # Vent indtil siden er færdigindlæst og overskriften er tilgængelig
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.TAG_NAME, 'h2')))
        
        # Scroll ned ad siden for at sikre, at alt indhold bliver indlæst
        time.sleep(2)  # Giv siden lidt tid til at indlæse
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Vent lidt for at sikre, at alt indlæses
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        # Hent overskriften (h2 er ofte brugt til hovedtitlen)
        title_element = driver.find_element(By.TAG_NAME, 'h2')
        title = title_element.text if title_element else 'No Title'

        # Hent datoen (tilpasset til den faktiske HTML-struktur på siden)
        date_element = driver.find_element(By.CLASS_NAME, 'article-byline__date')
        date = date_element.text if date_element else 'No Date'

        # Prøv først at hente hele artiklen ved at kombinere elementer fra forskellige sektioner
        try:
            intro_paragraph = driver.find_element(By.CSS_SELECTOR, 'p.article-header__intro').text
        except:
            intro_paragraph = ""

        # Prøv at hente teksten fra "articleBody"
        try:
            article_body = driver.find_element(By.ID, "articleBody")
            inner_html = article_body.get_attribute('innerHTML')
            soup = BeautifulSoup(inner_html, 'html.parser')
            paragraphs = soup.find_all('p')
            article_text = "\n".join([paragraph.get_text() for paragraph in paragraphs])
        except:
            article_text = ""

        # Kombinér intro-tekst og artikel-tekst fra "articleBody"
        full_article_text = f"{intro_paragraph}\n{article_text}" if intro_paragraph else article_text

        # Hvis teksten stadig er mangelfuld, prøv at finde alternative sektioner (som i HTML1)
        if not full_article_text or len(full_article_text) < 100:  # Brug en passende tærskel
            alternative_sections = driver.find_elements(By.CSS_SELECTOR, 'div.article__section p')
            if alternative_sections:
                alternative_text = "\n".join([section.text for section in alternative_sections])
                if alternative_text:
                    full_article_text = f"{intro_paragraph}\n{alternative_text}" if intro_paragraph else alternative_text
        
        # Tilføj detaljerne til listen
        articles.append({
            'URL': link,
            'Title': title,
            'Date': date,
            'Content': full_article_text
        })

    except TimeoutException:
        print(f"Timeout while trying to load the page: {link}")
    except Exception as e:
        print(f"An error occurred for {link}: {e}")

# Luk driveren, når alle sider er blevet besøgt
driver.quit()

# Konverter listen af artikeldetaljer til en pandas DataFrame
df = pd.DataFrame(articles)

  0%|▎                                                                            | 34/9167 [05:00<38:18:45, 15.10s/it]

Timeout while trying to load the page: https://www.berlingske.dk/virksomheder/kaempekrise-og-slagsmaal-om-milliardregning-stopper-metrobyggeri


  3%|██▋                                                                         | 318/9167 [41:57<36:19:31, 14.78s/it]

Timeout while trying to load the page: http://www.business.dk/aktier/live


  9%|██████▉                                                                   | 863/9167 [1:53:11<34:54:57, 15.14s/it]

...

 89%|████████████████████████████████████████████████████████████████▊        | 8133/9167 [18:20:10<4:19:55, 15.08s/it]

Timeout while trying to load the page: http://www.radio24syv.dk/aktier/tips-til-investering


 89%|████████████████████████████████████████████████████████████████▊        | 8144/9167 [18:21:54<4:12:12, 14.79s/it]

Timeout while trying to load the page: https://www.berlingske.dk/business/her-er-noeglepersonerne-omkring-lego


100%|███████████████████████████████████████████████████████████████████████████| 9167/9167 [20:39:13<00:00,  8.11s/it]


In [7]:
# Juster Pandas-indstillinger for at vise hele indholdet i 'Content' kolonnen
pd.set_option('display.max_colwidth', None)

# Vis DataFrame
df

URL  \
0                         https://www.berlingske.dk/virksomheder/dsv-vi-arbejder-konstant-paa-opkoeb   
1     https://www.berlingske.dk/virksomheder/finans-tv-graesk-zorbas-paa-det-europaeiske-aktiemarked   
2                 https://www.berlingske.dk/aktier/bogen-der-ryster-wall-street-og-storsaelger-i-usa   
3                          https://www.berlingske.dk/aktier/grus-i-maskineriet-til-fls-aktionaererne   
4                                https://www.berlingske.dk/aktier/sverige-oedelagde-dagen-for-nordea   
...                                                                                              ...   
9133                           https://www.berlingske.dk/business/euro-faar-kloe-efter-italiensk-nej   
9134                           https://www.berlingske.dk/aktier/usikkerhed-om-retning-fra-boersstart   
9135    https://www.berlingske.dk/virksomheder/gasfelt-er-afgoerende-for-maersks-fremtid-i-nordsoeen   
9136    https://www.berlingske.dk/aktier/kvindelig-chefoekonom-lad-bare-manden-styre-investeringerne   
9137                            https://www.berlingske.dk/virksomheder/nye-vaaben-klar-i-mobilkrigen   

                                                            Title  \
0                              DSV: Vi arbejder konstant på opkøb   
1           Finans-TV: Græsk zorbas på det europæiske aktiemarked   
2               Bogen, der ryster Wall Street og storsælger i USA   
3                         Grus i maskineriet til FLS-aktionærerne   
4                               Sverige ødelagde dagen for Nordea   
...                                                           ...   
9133                             Euro får klø efter italiensk nej   
9134                          Usikkerhed om retning fra børsstart   
9135           Gasfelt er afgørende for Mærsks fremtid i Nordsøen   
9136  Kvindelig cheføkonom: Lad bare manden styre investeringerne   
9137                                 Nye våben klar i mobilkrigen   

                                         Date  \
0       Fredag d. 06. februar 2015, kl. 09.32   
1          Fredag d. 10. juli 2015, kl. 13.10   
2           Mandag d. 05. maj 2014, kl. 10.48   
3           Fredag d. 17. maj 2013, kl. 17.07   
4     Onsdag d. 25. september 2013, kl. 18.11   
...                                       ...   
9133   Mandag d. 05. december 2016, kl. 07.01   
9134    Onsdag d. 16. oktober 2013, kl. 08.52   
9135        Onsdag d. 11. maj 2016, kl. 08.19   
9136    Torsdag d. 14. august 2014, kl. 17.00   
9137    Mandag d. 29. februar 2016, kl. 06.00   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [8]:
# Gem DataFrame til en CSV-fil, hvis ønsket
df.to_csv('berlingske_c20_Endelig.csv', index=False)